# Comparison with article

In [1]:
from coupled_cluster.schemes.ccd_sparse import CoupledClusterDoublesSparse
from coupled_cluster.hartree_fock.scf_rhf import scf_rhf
from coupled_cluster.matrix_elements.generate_matrices import (
    get_one_body_elements, get_coulomb_elements,
    get_antisymmetrized_elements, add_spin_to_one_body_elements,
    get_one_body_elements_spin
)
from coupled_cluster.matrix_elements.index_map import (
    generate_index_map, IndexMap
)
from coupled_cluster.hartree_fock.basis_transformation import (
    transform_one_body_elements, transform_two_body_elements
)

import numpy as np
import time
import os

In [2]:
file_path = os.path.join("..", "dat")
filename = os.path.join(file_path, "coulomb_{0}.pkl")

num_shells = 8
generate_index_map(num_shells)

omega = 1.0
l = IndexMap.shell_arr[-1]
n = 12
theta = 0.8

filename = filename.format(l)

print ("""
w = {0},
l = {1},
n = {2},
theta = {3},
filename = {4}
""".format(omega, l, n, theta, filename))


w = 1.0,
l = 72,
n = 12,
theta = 0.8,
filename = ../dat/coulomb_72.pkl



In [3]:
h = omega * get_one_body_elements(l)
t0 = time.time()
u = np.sqrt(omega) * get_coulomb_elements(l, filename=filename, tol=1e-12)
t1 = time.time()
print ("Time spent creating Coulomb elements: {0} sec".format(t1 - t0)) 

t0 = time.time()
c, energy = scf_rhf(h.todense(), u, np.eye(l//2), n//2)
t1 = time.time()
print ("Time spent in SCF RHF: {0} sec".format(t1 - t0))

print ("\tRHF Energy: {0}".format(energy))

hi = transform_one_body_elements(h, c)
t0 = time.time()
oi = transform_two_body_elements(u, c)
t1 = time.time()
print ("Time spent transforming two body elements: {0} sec".format(t1 - t0))

_h = add_spin_to_one_body_elements(hi, l)
t0 = time.time()
_u = get_antisymmetrized_elements(l, oi=oi, tol=1e-12)
t1 = time.time()
print ("Time spent antisymmetrizing two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd_hf = CoupledClusterDoublesSparse(_h, _u, n)
t1 = time.time()
print ("Time spent setting up CCD code with HF basis: {0} sec".format(t1 - t0))
t0 = time.time()
energy, iterations = ccd_hf.compute_energy(tol=1e-4, theta=theta)
t1 = time.time()
print ("Time spent computing CCD energy with HF basis: {0} sec".format(t1 - t0))
print ("\tCCD (HF) Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))

__h = omega * get_one_body_elements_spin(l)
t0 = time.time()
__u = np.sqrt(omega) * get_antisymmetrized_elements(l, filename=filename)
t1 = time.time()
print ("Time spent getting antisymmetric two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd = CoupledClusterDoublesSparse(__h, __u, n)
t1 = time.time()
print ("Time spent setting up CCD code with HO basis: {0} sec".format(t1 - t0))
t0 = time.time()
energy, iterations = ccd.compute_energy(theta=theta)
t1 = time.time()
print ("Time spent computing CCD energy with HO basis: {0} sec".format(t1 - t0))
print ("\tCCD Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))

Time spent creating Coulomb elements: 31.046775102615356 sec
Time spent in SCF RHF: 0.7470028400421143 sec
	RHF Energy: 66.92309448217796
Time spent transforming two body elements: 0.461284875869751 sec
Time spent antisymmetrizing two body elements: 3.0064141750335693 sec
Time spent setting up CCD code with HF basis: 2.5227301120758057 sec
Time spent computing CCD energy with HF basis: 154.82737588882446 sec
	CCD (HF) Energy: 65.97274237528121
	Iterations: 32
	Second/iteration: 4.8383554965257645
Time spent getting antisymmetric two body elements: 0.81784987449646 sec
Time spent setting up CCD code with HO basis: 0.9535810947418213 sec


/Users/Schoyen/anaconda3/lib/python3.6/site-packages/sparse/utils.py:190: RuntimeWarning: invalid value encountered in subtract
  return self.func(*totargs, **kwargs)
/Users/Schoyen/anaconda3/lib/python3.6/site-packages/sparse/utils.py:190: RuntimeWarning: invalid value encountered in add
  return self.func(*totargs, **kwargs)


Time spent computing CCD energy with HO basis: 40.11674213409424 sec
	CCD Energy: nan
	Iterations: 63
	Second/iteration: 0.6367736846681625


In [4]:
print (h.density)
print (u.density)
print (_u.density)
print (_h.density)
print (__u.density)
print (__h.density)

0.027777777777777776
0.05720831428135955
0.10878022119341564
0.06172839506172839
0.021026234567901234
0.013888888888888888
